In [ ]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error, r2_score
import time

# Load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

total_vram_usage = 0
total_rmse = 0
total_r2 = 0
num_checks = 5

# CNN model
for epoch in range(num_checks):
    start_time = time.time()

    with tf.device('/GPU:0'):
        cnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=[])
        cnn_model.fit(x_train, y_train, epochs=1, verbose=2)

        y_pred = cnn_model.predict(x_test)

        rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_test, y_pred))))
        r2 = r2_score(y_test, y_pred)

        print(f"CNN Model, Epoch {epoch + 1}/5 - RMSE: {rmse:.4f}, R-squared: {r2:.4f}")

    end_time = time.time()

    gpu_info = !nvidia-smi
    vram_usage_line = next((line for line in gpu_info if 'MiB /' in line), None)

    if vram_usage_line:
        vram_used_str = vram_usage_line.split()[8]
        vram_used = int(vram_used_str[:-3])
        print(f"VRAM Usage: {vram_used} MiB")

        total_vram_usage += vram_used
        total_rmse += rmse
        total_r2 += r2

        time.sleep(5)

average_vram_usage = total_vram_usage / num_checks
average_rmse = total_rmse / num_checks
average_r2 = total_r2 / num_checks
print(f"Average VRAM Usage: {average_vram_usage} MiB")
print(f"Average RMSE: {average_rmse:.4f}")
print(f"Average R-squared: {average_r2:.4f}")


11490434/11490434 [==============================] - 0s 0us/step
1 Physical GPUs, 1 Logical GPUs
1875/1875 - 13s - loss: 1.5287 - 13s/epoch - 7ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 1/5 - RMSE: 3.9857, R-squared: 0.8914
VRAM Usage: 2187 MiB
1875/1875 - 5s - loss: 0.7996 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 2/5 - RMSE: 4.0131, R-squared: 0.9186
VRAM Usage: 2187 MiB
1875/1875 - 5s - loss: 0.5816 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 3/5 - RMSE: 4.0186, R-squared: 0.9355
VRAM Usage: 2187 MiB
1875/1875 - 5s - loss: 0.4626 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 4/5 - RMSE: 4.0243, R-squared: 0.9382
VRAM Usage: 2187 MiB
1875/1875 - 5s - loss: 0.3856 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 5/5 - RMSE: 3.9903, R-squared: 0.9403
VRAM Us